In [6]:
import numpy as np
import matplotlib
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import time


import sys
sys.path += ['Package/']  ### Specify the directory where the Package is
from functions_for_DDCSBM import *

import warnings
warnings.filterwarnings("ignore")

## Figure 5

In [7]:
n = 300 # number of nodes
T = 3 # number of time-frames
n_clusters = 4 # number of communities
c_out = 2 # average number of connections between nodes in different communities
c = 6 # average degree
eta = 0.9 # label persistence
theta = np.random.uniform(3,10,n)**0 # vector theta
theta = theta/np.mean(theta) # E[theta_i] = 1
phi = np.mean(theta**2)
fluctuation = 0/n_clusters # fluctuations of off-diagonal elements of C (see documentation)
fraction = np.random.normal(1/n_clusters, 0/(4*n_clusters),n_clusters) # vector pi
fraction = fraction/np.sum(fraction) # \sum_i \pi_i = 1
C = matrix_C(c_out, c,fluctuation, fraction) # affinity matrix C


AT, Label = adjacency_matrix_series_DDCSBM(T, C, c, eta, theta,fraction) 

In [ ]:
t0 = time.time()
h = 0.6
xi = 0.8
w = B_spec(AT, n, xi, h, 'all')
y = 4.2
H = BH_matrix(AT,n,xi/y,h/y)
v = np.linalg.eigvalsh(H.A)
print('elapsed time : ' + str(time.time()-t0))

In [ ]:
fig = plt.figure(figsize = (16,5))

ax1 = plt.subplot2grid((5,10), (0,0), colspan=5, rowspan=5)
ax2 = plt.subplot2grid((5,10), (0,5), colspan=5, rowspan=5)

ax1.scatter(w[w.real < y].real,w[w.real < y].imag, color = 'white',edgecolor = 'r', s = 6)
ax1.scatter(w[w.real > y].real,w[w.real > y].imag, color = 'r',edgecolor = 'k')
M_T = matrix_M(T,xi,xi,c,phi,h)
rho = np.max(np.linalg.eigvals(M_T).real)
radius = np.sqrt(rho)
x = np.linspace(-radius,radius)
yv = np.sqrt(radius**2-x**2)
ax1.plot(x, yv, color = 'k', linewidth = 2)
ax1.plot(x, -yv, color = 'k', linewidth = 2)
ax1.axvline(y, color = 'b', linestyle = '--', label = 'y')
ax1.legend(fontsize = 14)


factor = 3
ax2.hist(v[v>0], bins = 100, edgecolor = 'k', color = 'indigo', alpha = 0.35)
list_of_lists = [list(v[v < 0]) for i in range(factor)]
flattened  = [val for sublist in list_of_lists for val in sublist]
ax2.hist(flattened, bins = 10, edgecolor = 'k', color = 'darkblue', alpha = 1)

ts = 18


ax1.set_xlabel('Real part', fontsize = 20)
ax1.set_ylabel('Imaginary part', fontsize = 20)
ax1.tick_params(axis="y", labelsize=ts)
ax1.tick_params(axis="x", labelsize=ts)

ax2.set_xlabel(r'$s(H_{\xi,h})$', fontsize = 20)
ax2.set_ylabel(r'$\nu[s(H_{\xi,h})]$', fontsize = 20, rotation = -90, labelpad = 25)
ax2.tick_params(axis="y", labelsize=ts)
ax2.tick_params(axis="x", labelsize=ts)

ax2.yaxis.set_label_position("right")
ax2.yaxis.tick_right()

    
plt.show();

## Figure 6

In [ ]:
n = 10000 # size of the network
T = 5 # number of frames
n_clusters = 2 # number of clusters

eta = 0.4 # label persistence
c = 10 # average degree
c_out = 4# c_out
c_in = 2*c-c_out # c_in
lamb = (c_in - c_out)/(c_in + c_out) # \lambda

C = np.array(([c_in, c_out], [c_out, c_in])) # matrix C
theta = np.random.uniform(3,10,n)**3 # vector theta (power law degree distribution)
theta = theta/np.mean(theta) # normalization of theta

phi = np.mean(theta**2) # phi
fraction = np.ones(n_clusters)/n_clusters

alpha_c = find_transition(T,eta) # critial value of the transition give T and eta
lambda_d = alpha_c/np.sqrt(c*phi)

xi1 = 0.2
h1 = 0.9

### Note

Here we are computing the $kT$ largest eigenvalues.To reproduce the result of the paper replace n_clusters*T with 150. In this plot we only compute the first $kT$ eigenvalues because the computation of the bulk eigenvalues sensibly slows down the simulation.

In [ ]:
t0 = time.time()
AT, Label = adjacency_matrix_series_DDCSBM(T, C, c, eta, theta,fraction) 
v1 = B_spec(AT, n, xi1, h1, n_clusters*T) 
print(time.time()-t0)

In [ ]:
t0 = time.time()
h2 = 0.7
xi2 = 0.4
v2 = B_spec(AT, n, xi2, h2, n_clusters*T)
print(time.time()-t0)

In [ ]:
fig = plt.figure(figsize = (18,9))

ax1 = plt.subplot2grid((4,4), (2,0), colspan=2, rowspan=2)
ax2 = plt.subplot2grid((4,4), (2,2), colspan=2, rowspan=2)


h = h1
xi = xi1

vrs = v1.real
vis = v1.imag

fs = 18
ts = 14

ax1.set_xlabel('Real part', fontsize = fs)
ax1.set_ylabel('Imaginary part', fontsize = fs)
ax1.tick_params(axis="x", labelsize=ts)
ax1.tick_params(axis="y", labelsize=ts)
ax1.tick_params(top='off', bottom='off', left='off', right='off', labelleft='off', labelbottom='on')

M1 =  matrix_M(T, 1, xi, c, phi, np.sqrt(h))
M2 =  matrix_M(T, lamb, xi, c, phi, np.sqrt(eta*h))
M3 =  matrix_M(T, xi, xi, c, phi, h)

info = np.linalg.eigvals(M2)
rho = np.linalg.eigvals(M1)
bulk = np.linalg.eigvals(M3)
idx = np.argsort(info)[::-1]
info = info[idx]
idx = np.argsort(rho)[::-1]
rho = rho[idx]
radius = np.sqrt(max(bulk.real))



x = np.linspace(0.8*radius,radius)
xp = np.linspace(0.7*radius,0.8*radius)
ax1.plot(x,np.sqrt(radius**2-x**2), color = 'k', linewidth = 1.5)
ax1.plot(x,-np.sqrt(radius**2-x**2), color = 'k', linewidth = 1.5, label = r'$\sqrt{\rho(M^{,,}_T)}$')
ax1.plot(xp,np.sqrt(radius**2-xp**2), color = 'k', linewidth = 1.5, linestyle = '--')
ax1.plot(xp,-np.sqrt(radius**2-xp**2), color = 'k', linewidth = 1.5, linestyle = '--')

for i in range(3*T):
    if np.abs(info[i].real) > radius:
        if info[i].imag < 10**(-4):
            ax1.axvline(info[i].real, color = 'blue', linewidth = 1, linestyle = '--')
        else: 
            ax1.scatter(info[i].real,info[i].imag, color = 'white', edgecolor = 'blue', s = 50)

    if np.abs(rho[i].real) > radius:
        if np.abs(rho[i].imag) < 10**(-4):
            ax1.axvline(rho[i].real, color = 'k', linewidth = 1, linestyle = '-.')
        else: 
            ax1.scatter(rho[i].real,rho[i].imag, color = 'white', edgecolor = 'k', s = 50)
  
ax1.scatter(vrs, vis, s = 15, edgecolor = 'r', color = 'white')

ax1.axvline(info[0], color = 'b', linewidth = 6, linestyle = '-', alpha = 0.2)
ax1.axvline(rho[0], color = 'k', linewidth = 6, linestyle = '-', alpha = 0.2)

ax1.plot(np.linspace(0.4,6.5,50), np.zeros(50), linewidth = 0.5, color = 'k')
ax1.set_xlim(0.8,5)


#######################################################################################################

h = h2
xi = xi2

vrb = v2.real
vib = v2.imag

fs = 18
ts = 14

ax2.set_xlabel('Real part', fontsize = fs)
ax2.tick_params(axis="x", labelsize=ts)
ax2.tick_params(axis="y", labelsize=0)
ax2.tick_params(top='off', bottom='off', left='off', right='off', labelleft='off', labelbottom='on')

M1 =  matrix_M(T, 1, xi, c, phi, np.sqrt(h))
M2 =  matrix_M(T, lamb, xi, c, phi, np.sqrt(eta*h))
M3 =  matrix_M(T, xi, xi, c, phi, h)

info = np.linalg.eigvals(M2)
rho = np.linalg.eigvals(M1)
bulk = np.linalg.eigvals(M3)
idx = np.argsort(info)[::-1]
info = info[idx]
idx = np.argsort(rho)[::-1]
rho = rho[idx]
radius = np.sqrt(max(bulk.real))



x = np.linspace(0.8*radius,radius)
xp = np.linspace(0.7*radius,0.8*radius)
ax2.plot(x,np.sqrt(radius**2-x**2), color = 'k', linewidth = 1.5)
ax2.plot(x,-np.sqrt(radius**2-x**2), color = 'k', linewidth = 1.5, label = r'$\sqrt{\rho(M^{,,}_T)}$')
ax2.plot(xp,np.sqrt(radius**2-xp**2), color = 'k', linewidth = 1.5, linestyle = '--')
ax2.plot(xp,-np.sqrt(radius**2-xp**2), color = 'k', linewidth = 1.5, linestyle = '--')

for i in range(3*T):
    if np.abs(info[i].real) > radius:
        if info[i].imag < 10**(-4):
            ax2.axvline(info[i].real, color = 'blue', linewidth = 1, linestyle = '--')
        else: 
            ax2.scatter(info[i].real,info[i].imag, color = 'white', edgecolor = 'blue', s = 50)

    if np.abs(rho[i].real) > radius:
        if np.abs(rho[i].imag) < 10**(-4):
            ax2.axvline(rho[i].real, color = 'k', linewidth = 1, linestyle = '-.')
        else: 
            ax2.scatter(rho[i].real,rho[i].imag, color = 'white', edgecolor = 'k', s = 50)
  
ax2.scatter(vrb, vib, s = 15, edgecolor = 'r', color = 'white')

ax2.axvline(info[0], color = 'b', linewidth = 6, linestyle = '-', alpha = 0.2)
ax2.axvline(rho[0], color = 'k', linewidth = 6, linestyle = '-', alpha = 0.2)

ax2.plot(np.linspace(0.4,10,50), np.zeros(50), linewidth = 0.5, color = 'k')
ax2.set_xlim(0.8,8)



# ##################################################################

plt.show();

## Figure 7

In [ ]:
n = 500 # size of the network
T = 2 # number of frames

eta = 0.4 # label persistence
c = 6 # average degree
c_out = 1. # c_out
c_in = 2*c-c_out # c_in
lamb = (c_in - c_out)/(c_in + c_out) # \lambda
fraction = np.ones(2)/2

C = np.array(([c_in, c_out], [c_out, c_in])) # matrix C
theta = np.ones(n)# vector theta (power law degree distribution)
phi = 1 # phi

alpha_c = find_transition(T,eta) # critial value of the transition give T and eta
lamb_d = alpha_c/np.sqrt(c*phi) # value of lambda at the transition 

In [ ]:
t0 = time.time()
AT, Label = adjacency_matrix_series_DDCSBM(T, C, c, eta, theta,fraction) # generate the series of adjacency matrices

v = B_spec(AT,n,lamb_d,eta, 'all') # matrix B_{\lambda_d,\eta}
vr = v.real
vi = v.imag

vBP = B_spec(AT,n,lamb,eta, 'all') # matrix B_{\lambda,\eta}
vrBP = vBP.real
viBP = vBP.imag


H  = BH_matrix(AT,n,lamb_d,eta) # matrix H_{\lambda_d, \eta}
w = np.linalg.eigvals(H.A)

H  = BH_matrix(AT,n,lamb,eta) # matrix H_{\lambda,\eta}
wBP = np.linalg.eigvals(H.A)
print(time.time()-t0)

In [ ]:
idx = np.argsort(vr)[::-1]
vr = vr[idx]
vi = vi[idx]

idx = np.argsort(vrBP)[::-1]
vrBP = vrBP[idx]
viBP = viBP[idx]

fig = plt.figure(figsize = (18,16))

ax1 = plt.subplot2grid((8,8), (2,0), colspan=2, rowspan=2)
ax3 = plt.subplot2grid((8,8), (2,2), colspan=2, rowspan=2)
ax2 = plt.subplot2grid((8,8), (2,4), colspan=2, rowspan=2)
ax4 = plt.subplot2grid((8,8), (2,6), colspan=2, rowspan=2)



fs = 18
ts = 14

ax1.set_xlabel('Real part', fontsize = fs)
ax1.set_ylabel('Imaginary part', fontsize = fs)
ax1.tick_params(axis="x", labelsize=0)
ax1.tick_params(axis="y", labelsize=0)
ax1.tick_params(top='off', bottom='off', left='off', right='off', labelleft='off', labelbottom='on')
ax1.scatter(vrBP[4:], viBP[4:], s = 7, edgecolor = 'r', color = 'white')
ax1.scatter(vrBP[:4], viBP[:4], s = 15, edgecolor = 'b', color = 'white')
M_T = matrix_M(T,lamb,lamb,c,phi,eta)
radius = np.sqrt(np.max(np.linalg.eigvals(M_T).real))
x = np.linspace(-radius,radius)
ax1.plot(x,np.sqrt(radius**2-x**2), color = 'k', linewidth = 2.)
ax1.plot(x,-np.sqrt(radius**2-x**2), color = 'k', linewidth = 2., label = r'$\sqrt{\rho(M_T(\alpha))}$')
ax1.axvline(1, color = 'darkgreen', linestyle = '--', linewidth = 1.5)

# ################################################################################



ax2.set_xlabel(r'$s(H_{\lambda,\eta})$', fontsize = fs)
ax2.tick_params(axis="x", labelsize=0)
ax2.tick_params(axis="y", labelsize=0)
ax2.tick_params(top='off', bottom='off', left='off', right='off', labelleft='off', labelbottom='on')

factor = 3
wBP = np.sort(wBP)
list_of_lists = [list(wBP[:4]) for i in range(factor)]
flattened  = [val for sublist in list_of_lists for val in sublist]
ax2.hist(flattened, bins = 5, edgecolor = 'k', color = 'darkblue', label = 'Informative modes')
ax2.hist(wBP[4:], bins = 70, edgecolor = 'k', color = 'purple', alpha = 0.4, label = 'Bulk')
ax2.axvline(0, color = 'k', linestyle = '--')
ax2.set_xlim(-10,40)



# ######################################################################################

ax3.set_xlabel('Real part', fontsize = fs)
ax3.set_ylabel('Imaginary part', fontsize = 0)
ax3.tick_params(axis="x", labelsize=0)
ax3.tick_params(axis="y", labelsize=0)
ax3.tick_params(top='off', bottom='off', left='off', right='off', labelleft='off', labelbottom='on')
ax3.scatter(vr[4:], vi[4:], s = 5, edgecolor = 'r', color = 'white')
ax3.scatter(vr[:4], vi[:4], s = 15, edgecolor = 'b', color = 'white')
radius = 1
x = np.linspace(-radius,radius)
ax3.plot(x,np.sqrt(radius**2-x**2), color = 'k', linewidth = 2.)
ax3.plot(x,-np.sqrt(radius**2-x**2), color = 'k', linewidth = 2., label = r'$\sqrt{\rho(M_T(\alpha))}$')
ax3.axvline(1, color = 'darkgreen', linestyle = '--', linewidth = 1.5)


# ########################################################################################


ax4.set_xlabel(r'$s(H_{\lambda_d,\eta})$', fontsize = fs)
ax4.tick_params(axis="x", labelsize=0)
ax4.tick_params(axis="y", labelsize=0)
ax4.tick_params(top='off', bottom='off', left='off', right='off', labelleft='off', labelbottom='on')

factor = 3
w = np.sort(w)
list_of_lists = [list(w[:4]) for i in range(factor)]
flattened  = [val for sublist in list_of_lists for val in sublist]
ax4.hist(flattened, bins = 12, edgecolor = 'k', color = 'darkblue', label = 'Informative modes')
ax4.hist(w[6:], bins = 60, edgecolor = 'k', color = 'purple', alpha = 0.4, label = 'Bulk')
ax4.axvline(0, color = 'k', linestyle = '--')


# ###################################################################

plt.show();